# Example documenting Synergia Nonlinear Insert Creation

This notebook shows how to create the series of `nllens` elements in MAD-X/Synergia in such a way as to constitute a complete nonlinear insert. The intention with this development is to permit users to define the basic composition of a nonlinear insert, and place it anywhere in the lattice that satisfies the appropriate lattice functions. 

The resulting `NonlinearInsert` class, shown below, does not rely on the rssynergia repository, nor any other Synergia dependencies, but is designed to facilitate the specification of nonlinear lattice designs in Sirepo. Future updates, for example the `validate_sequence()` function, will make use of code-specific helpers to grab lattice functions for processing. The `NonlinearInsert` class has been added to the `rsbeams` repository in [rsbeams/rslattice/nonlinear.py](https://github.com/radiasoft/rsbeams/blob/master/rsbeams/rslattice/nonlinear.py).

Creation of the nonlinear insert requires the following specifications:

1. Number of insert segments - the IOTA lattice uses 20 segments so we will default to 20.
2. The nonlinear aperture parameter c - the IOTA lattice uses 0.01
3. The nonlinear strength parameter t - this is a free parameter, but we often use values between 0.1 and 0.4

**Note:** In order to meet the requirements of integrability, the insert must be placed in a drift for which the beta functions are symmetric and, and the insert parameters vary as a function of the phase advance through the insert and c-parameter. What this means is that a given lattice design may only accomodate one specific insert type (but with variable nonlinear strength). However, to streamline the user interface, we will ignore these requirements for now, and simply permit users to arbitrarily design and place these nonlinear inserts.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

/home/vagrant/.pyenv/versions/py2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from rsbeams.rsptcls import bunch
from rsbeams.rslattice import nonlinear

## Nonlinear Element Specification

#### Basic relationships between entrance, central betas and the length/aperture/phase advance of the element.

In [4]:
# Relationship between phase advance and length:
mu0 = 0.3
l0 = 1.8
cval = 0.03
tval = 0.4
f0=l0/4.0*(1.0+1.0/np.tan(np.pi*mu0)**2)
print "Focal length is {}m".format(f0)
betae=l0/np.sqrt(1.0-(1.0-l0/2.0/f0)**2)
print "Beta values at entrance are {}m".format(betae)
betas=l0*(1-l0/4.0/f0)/np.sqrt(1.0-(1.0-l0/2.0/f0)**2);
print "Beta value at center is {}m".format(betas)

Focal length is 0.68753882025m
Beta values at entrance are 1.89263200363m
Beta value at center is 0.653888275205m


## Nonlinear Insert Python Class

In [5]:
class NonlinearInsert(object):
    """
    Class for generating and manipulating a nonlinear insert for use in the nonlinear integrable optics.
    Following the presciptions of the elliptic scheme defined by: 
       -V. Danilov and S. Nagaitsev. Phys. Rev. ST Accel. Beams 13, 084002 (2010)
       -https://journals.aps.org/prab/abstract/10.1103/PhysRevSTAB.13.084002    
    
    Instantiation of the nonlinear insert requires element requires specification of the length, phase, 
    aperture, strength, and number of slices comprising the insert. The required Twiss functions can be
    computed from this information, and hence insertion of the element into a lattice can be done by 
    comparing the requirements to the Twiss parameters obtained from the lattice.
    
    Attributes:
        length (float): the length of the nonlinear insert in meters
        phase (float): the phase advance modulo 2pi through the nonlinear insert
        t (float): the dimensionless nonlinear strength parameter. Defaults to 0.1
        c (float): the nonlinear aperture parameter (m^-1/2), defining poles in the x-axis. Defaults to 0.01.
        num_slices (int): the number of piecewise constant segements used to construct the insert
        
        s_vals (ndArray): array of relative s-values providing the center of each segment of the nonlinear insert
        knll (ndArray): array of gradient values for each nonlinear segment element (nllens)
        cnll (ndArray): array of aperture parameters for each nonlinear segment element (nllens)
       
    """
    
    def __init__(self, length, phase, t = 0.1, c = 0.01, num_slices = 20):
    
        """
        Arguments:
            length (float): the length of the nonlinear insert in meters
            phase (float): the phase advance modulo 2pi through the nonlinear insert
            t (float): the dimensionless nonlinear strength parameter. Defaults to 0.1
            c (float): the nonlinear aperture parameter (m^-1/2), defining poles in the x-axis. Defaults to 0.01.
            num_slices (int): the number of piecewise constant segements used to construct the insert
        """
        self.length = length
        self.phase = phase
        self.t = t
        self._c = c
        self.num_slices = num_slices
    
        
    #Define c property which maintains a positive definite value
    @property
    def c(self):
        return self._c
    @c.setter
    def c(self, cval):
        if cval < 0:
            raise ValueError("Aperture parameter c must be larger than 0.")     
        self._c = c
        
    
    def generate_sequence(self):
        """Generates arrays containing the knll and cnll values for each nllens element"""
        
        #Define the focal length of the insert using the phase advance and length
        f0 = self.length/4.0*(1.0+1.0/np.tan(np.pi*self.phase)**2)
        
        #define array of s-values
        start = (self.length/self.num_slices)*0.5
        end = self.length - start
        #Make an attribute as they could be useful for constructing the mad-x sequence
        self.s_vals = np.linspace(start,end,self.num_slices) 
        
        #set the initial beta value to help compare to lattice functions
        self.beta0 = self.length*(1.-0.0*(self.length)/self.length/f0)/np.sqrt(1.0-(1.0-self.length/2.0/f0)**2)
        
        #set the beta functions as an attribute for comparing against other lattice functions
        bn = self.length*(1-self.s_vals*(self.length-self.s_vals)/self.length/f0)/np.sqrt(1.0-(1.0-self.length/2.0/f0)**2)
        self.betas = bn
        
        knn = self.t*self.length/self.num_slices/bn**2
        cnll = self.c*np.sqrt(bn)
        knll = knn*cnll**2
        
        #Now set the knll and cnll parameters for each nllens object
        self.knll = knll
        self.cnll = cnll
        
    def validate_sequence(self, beta_values):
        """
        Checks the predicted beta functions for the specified sequence against known lattice functions.
        This function is not currently implemented.
        
        Arguments:
            beta_values: an array of beta x/y values for the underlying "bare" lattice
            
        Returns:
            check: a boolean signifying that the lattice correctly permits the sequence defined in the lattice
        """
        
        return
        
            
    def create_madx(self):
        """
        Return a sequence of madx elements representing the insert, represented as strings:
        
            ["elem 1 string", "elem 2 string", ...]
        
        Returns:
            MADX_elements: list of strings describing nllens elements needed to construct inset
        """
        
        MADX_elements = []
        
        for ind in range(len(self.knll)):
            #Loop through element and construct nllens with strengths and apertures
            elem = "nllens, knll = {}, cnll = {};".format(self.knll[ind], self.cnll[ind])
            MADX_elements.append(elem)
            
        return MADX_elements

#### Simple Example

This example reproduces the NL insert for standard IOTA parameters:
    - length = 1.8m
    - phase = 0.3 (2 $\pi$)
    - c = 0.01 m$^{1/2}$
    - t = 0.4

In [6]:
l0 = 1.8
mu0 = 0.3
c = 0.01
t = 0.4

#Import the version of the class from the nonlinear.py module
testNLI = nonlinear.NonlinearInsert(l0, mu0, t, c)
testNLI.generate_sequence()
testNLI.create_madx()

['nllens, knll = 2.03176954998e-06, cnll = 0.0133111024716;',
 'nllens, knll = 2.32425918167e-06, cnll = 0.0124454032437;',
 'nllens, knll = 2.66532093143e-06, cnll = 0.0116218829044;',
 'nllens, knll = 3.0579546094e-06, cnll = 0.0108501497707;',
 'nllens, knll = 3.49987469896e-06, cnll = 0.0101420326023;',
 'nllens, knll = 3.97907061816e-06, cnll = 0.00951174999782;',
 'nllens, knll = 4.46852825639e-06, cnll = 0.00897571379856;',
 'nllens, knll = 4.92267463887e-06, cnll = 0.00855166509068;',
 'nllens, knll = 5.28045052801e-06, cnll = 0.0082568759776;',
 'nllens, knll = 5.47957603728e-06, cnll = 0.00810546195183;',
 'nllens, knll = 5.47957603728e-06, cnll = 0.00810546195183;',
 'nllens, knll = 5.28045052801e-06, cnll = 0.0082568759776;',
 'nllens, knll = 4.92267463887e-06, cnll = 0.00855166509068;',
 'nllens, knll = 4.46852825639e-06, cnll = 0.00897571379856;',
 'nllens, knll = 3.97907061816e-06, cnll = 0.00951174999782;',
 'nllens, knll = 3.49987469896e-06, cnll = 0.0101420326023;',
 